<a href="https://colab.research.google.com/github/paritosh-Shukla24/Open_ai_agents/blob/main/YT_01_Learning_LangGraph_Agent_Executor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01. Learning LangGraph - Agent Executor

In [1]:
!pip install --quiet -U langchain langchain_openai langgraph langchainhub langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

modified from https://github.com/langchain-ai/langgraph/blob/main/examples/agent_executor/base.ipynb

In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "LangGraph_01"

## Making the GraphState

In [4]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
   # The input string
   input: str
   # The list of previous messages in the conversation
   chat_history: list[BaseMessage]
   # The outcome of a given call to the agent
   # Needs `None` as a valid type, since this is what this will start as
   agent_outcome: Union[AgentAction, AgentFinish, None]
   # List of actions and corresponding observations
   # Here we annotate this with `operator.add` to indicate that operations to
   # this state should be ADDED to the existing values (not overwrite it)
   intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

## Custom Tools

**Tools**  

Tools are interfaces that an agent can use to interact with the world. They combine a few things:



1.   The name of the tool
2.   A description of what the tool is
3.   JSON schema of what the inputs to the tool are
4.   The function to call


Whether the result of a tool should be returned directly to the user

In [5]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool

In [6]:
import random

@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
  """Returns the input as all lower case."""
  return input.lower()

@tool("random_number", return_direct=True)
def random_number_maker(input:str) -> str:
    """Returns a random number between 0-100."""
    return random.randint(0, 100)

tools = [to_lower_case,random_number_maker]

In [7]:
random_number_maker.run('random')

58

In [8]:
to_lower_case.run('SAM')

'sam'

## Agent - with new create_open_ai

In [9]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm,
                                               tools,
                                               prompt)

In [10]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlace

In [11]:
prompt.get_prompts()

[ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlac

In [12]:
inputs = {"input": "give me a random number and then write in words and make it lower case.",
          "chat_history": [],
          "intermediate_steps":[]}

agent_outcome = agent_runnable.invoke(inputs)

In [13]:
agent_outcome

AgentActionMessageLog(tool='random_number', tool_input={}, log='\nInvoking: `random_number` with `{}`\n\n\n', message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_44132a4de3'}, id='run-b21ef916-e1fa-4383-bc1e-57ed95cc39c2-0')])

In [14]:
type(agent_runnable)

langchain_core.runnables.base.RunnableSequence

## Nodes

In [15]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)

In [16]:
# Define the agent/graph
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

# Define the function to execute tools
def execute_tools(data):
    # Get the most recent agent_outcome - this is the key added in the `agent` above
    agent_action = data['agent_outcome']
    # Execute the tool
    output = tool_executor.invoke(agent_action)
    print(f"The agent action is {agent_action}")
    print(f"The tool result is: {output}")
    # Return the output
    return {"intermediate_steps": [(agent_action, str(output))]}

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(data['agent_outcome'], AgentFinish):
        return "end"
    # Otherwise, an AgentAction is returned
    # Here we return `continue` string
    # This will be used when setting up the graph to define the flow
    else:
        return "continue"

## Define the graph

In [17]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [18]:
workflow.branches

defaultdict(dict,
            {'agent': {'should_continue': Branch(path=should_continue(recurse=True), ends={'continue': 'action', 'end': '__end__'}, then=None)}})

In [19]:
workflow.nodes, workflow.edges

({'agent': agent(recurse=True), 'action': action(recurse=True)},
 {('__start__', 'agent'), ('action', 'agent')})

In [20]:
workflow.channels #['intermediate_steps']

{'input': <langgraph.channels.last_value.LastValue at 0x78616b8c7ac0>,
 'chat_history': <langgraph.channels.last_value.LastValue at 0x786169f4a0e0>,
 'agent_outcome': <langgraph.channels.last_value.LastValue at 0x786169f4a860>,
 'intermediate_steps': <langgraph.channels.binop.BinaryOperatorAggregate at 0x786169f49bd0>}

In [21]:
inputs = {"input": "give me a random number and then write in words and make it lower case.", "chat_history": []}
for s in app.stream(inputs):
    print(list(s.values())[0])
    print("----")

{'agent_outcome': AgentActionMessageLog(tool='random_number', tool_input={'input': 'random'}, log="\nInvoking: `random_number` with `{'input': 'random'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_44132a4de3'}, id='run-b27df48b-766f-411e-b759-051dac3f8c7a-0')])}
----
The agent action is tool='random_number' tool_input={'input': 'random'} log="\nInvoking: `random_number` with `{'input': 'random'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_44132a4de3'}, id='run-b27df48b-766f-411e-b759-051dac3f8c7a-0')]
The tool result is: 11
{'intermediate_steps': [(AgentActionMessage

In [22]:
inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

output = app.invoke(inputs)

The agent action is tool='random_number' tool_input={'input': 'random'} log="\nInvoking: `random_number` with `{'input': 'random'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_44132a4de3'}, id='run-595bdf6a-26e8-4cbc-974f-ae39ad3e266c-0')]
The tool result is: 86
The agent action is tool='lower_case' tool_input={'input': 'Eighty-six'} log="\nInvoking: `lower_case` with `{'input': 'Eighty-six'}`\n\n\n" message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"Eighty-six"}', 'name': 'lower_case'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_44132a4de3'}, id='run-f969243f-d916-49ef-8698-d7fdccc1f087-0')]
The tool result is: eighty-six


In [23]:
output.get("agent_outcome").return_values['output']

'The random number is 86, and in words, it is "eighty-six" in lower case.'

In [24]:
output.get("intermediate_steps")

[(AgentActionMessageLog(tool='random_number', tool_input={'input': 'random'}, log="\nInvoking: `random_number` with `{'input': 'random'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_44132a4de3'}, id='run-595bdf6a-26e8-4cbc-974f-ae39ad3e266c-0')]),
  '86'),
 (AgentActionMessageLog(tool='lower_case', tool_input={'input': 'Eighty-six'}, log="\nInvoking: `lower_case` with `{'input': 'Eighty-six'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"Eighty-six"}', 'name': 'lower_case'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_44132a4de3'}, id='run-f969243f-d916-49ef-8698-d7fdccc1f087-0')]),
  'eighty-six')]

In [25]:
inputs = {"input": "does it get cold in SF in Jan?", "chat_history": []}

output = app.invoke(inputs)

In [26]:
output.get("agent_outcome").return_values['output']


"Yes, San Francisco can get cold in January. The average high temperature in January is around 57°F (14°C), and the average low temperature is around 46°F (8°C). It's always a good idea to check the weather forecast closer to your planned visit to San Francisco."

In [27]:
output.get("intermediate_steps")

[]